In [ ]:
%reload_ext autoreload
%autoreload 2
from importlib import reload

import numpy as np
import matplotlib.pyplot as plt
import matplotlib.cm as cm
import h5py
import matplotlib as mpl
from tqdm import tqdm


from holodeck import plot, detstats
import holodeck.anisotropy as anis
from holodeck.constants import YR, MSOL, MPC
import holodeck as holo

import hasasia.sim as hsim
import os


In [ ]:
SHAPE = None
NREALS = 500
# NREALS = 20
NFREQS = 40
NLOUDEST = 10
BGL = 1

BUILD_ARRAYS = False
SAVEFIG = True
TOL=0.01
MAXBADS=5

NVARS = 21
# NVARS = 6

NPSRS = 40
NSKIES = 100
# NSKIES = 15

SHOW_GW=True
ALL_TARGETS = ['hard_time', 'gsmf_phi0', 'gsmf_mchar0_log10', 'mmb_mamp_log10', 'mmb_scatter_dex', 'hard_gamma_inner']

In [ ]:
cmap_Blues = plot.truncate_colormap('Blues', 0.4, 1)
cmap_Greens = plot.truncate_colormap('Greens', 0.4, 1)
cmap_Oranges = plot.truncate_colormap('Oranges', 0.4, 1)


In [ ]:
data, params, dsdat = detstats.get_data(target='hard_time',
        gw_only=False, red2white=None, red_gamma=None,
        nloudest=10, bg_nloudest=1, 
        gsc_flag=False, dsc_flag=False, divide_flag=False)


In [ ]:
dd = [10]
lmax=8
moll_hc10, Cl10 = anis.sph_harm_from_hc(
    data[10]['hc_ss'], data[10]['hc_bg'], nside=8, lmax=lmax
)

# Cl in shape (F, R, lmax+1)

In [ ]:
lmax=8

# Showing all harmonics looks the same

In [ ]:
xx = data[0]['fobs_cents'] * YR
colors = ['#0a5da4', '#00ba44', '#ff9503', '#fe2c02', '#845b98', '#474747']


xlabel = plot.LABEL_GW_FREQUENCY_YR
ylabel = plot.LABEL_CLC0
fig, ax = plot.figax_single(
)
ax.set_xlabel(xlabel)
ax.set_ylabel(ylabel)


for ll in range(1,7):
    plot.draw_med_conf_color(ax, xx, Cl[:,:,ll]/Cl[:,:,0], color=colors[ll-1])



# For several different vars

In [ ]:
parvars = [0,10,20]
lmax=8

yy_cl = [] #npars len array of (F,R,l=1) arrays
for pv in parvars:
    moll_hc, Cl = anis.sph_harm_from_hc(
        data[pv]['hc_ss'], data[pv]['hc_bg'], nside=8, lmax=lmax
    )
    yy_cl.append(Cl)

# Cl in shape (F, R, lmax+1)


 Middle again

In [ ]:
xx = data[0]['fobs_cents'] * YR
colors = ['#0a5da4', '#00ba44', '#ff9503', '#fe2c02', '#845b98', '#474747']


xlabel = plot.LABEL_GW_FREQUENCY_YR
ylabel = plot.LABEL_CLC0
fig, ax = plot.figax_single(
)
ax.set_xlabel(xlabel)
ax.set_ylabel(ylabel)


for ll in range(1,7):
    plot.draw_med_conf_color(ax, xx, yy_cl[1][:,:,ll]/yy_cl[1][:,:,0], color=colors[ll-1])



In [ ]:
for pp in range(3):
    print(holo.utils.stats(yy_cl[pp]))

In [ ]:


xx = data[0]['fobs_cents'] * YR
colors = cmap_Blues(np.linspace(0,1,len(parvars)))
target='hard_time'


xlabel = plot.LABEL_GW_FREQUENCY_YR
ylabel = plot.LABEL_CLC0
fig, ax = plot.figax_single(
)
ax.set_xlabel(xlabel)
ax.set_ylabel(ylabel)

ll=1
handles = []
for vv, var in enumerate(parvars):
    yy = yy_cl[vv][:,:,ll]/yy_cl[vv][:,:,0] # F,R
    pp = 68
    med, *conf = np.percentile(yy, (50, 50-pp/2, 50+pp/2), axis=-1)
    hh, = ax.plot(xx, med, label=params[var][target], color=colors[vv])
    handles.append(hh)
    ax.fill_between(xx, *conf, alpha=0.25, color=colors[vv])

ax.legend(handles=handles, loc='lower right', title = plot.PARAM_KEYS[target], ncols=3)

In [ ]:
def build_anis_arrays(
        target, nreals=500, nskies=100, 
        gw_only=False, red2white=None, red_gamma=None, 
        nloudest=10, bgl=10, 
        gsc_flag=False, dsc_flag=False, divide_flag=False,
        figpath = '/Users/emigardiner/GWs/holodeck/output/anatomy_redz/figdata/ratio',
        parvars = [0,10,20] 

        ):
    
    data, params, dsdat = detstats.get_data(target,
        gw_only=gw_only, red2white=red2white, red_gamma=red_gamma,
        nloudest=10, bg_nloudest=10, 
        gsc_flag=gsc_flag, dsc_flag=dsc_flag, divide_flag=divide_flag)


    lmax=8

    yy_cl = [] # PV len array of (F,R,l=1) arrays
    params_cl = []
    xx_fobs = data[0]['fobs_cents']
    for var in parvars:
        _, Cl = anis.sph_harm_from_hc(
            data[var]['hc_ss'], data[var]['hc_bg'], nside=8, lmax=lmax
        )
        yy_cl.append(Cl)
        params_cl.append(params[var])
    return yy_cl, xx_fobs, params_cl


# Vary hard_nu_inner

In [ ]:
target = 'hard_gamma_inner'
parvars = [0,10,20]
yy_cl, xx_fobs, params_cl = build_anis_arrays(target, parvars=parvars)

In [ ]:
xx = data[0]['fobs_cents'] * YR
colors = cmap_Blues(np.linspace(0,1,len(parvars)))


xlabel = plot.LABEL_GW_FREQUENCY_YR
ylabel = plot.LABEL_CLC0
fig, ax = plot.figax_single(
)
ax.set_xlabel(xlabel)
ax.set_ylabel(ylabel)

ll=1
handles = []
for vv, var in enumerate(parvars):
    yy = yy_cl[vv][:,:,ll]/yy_cl[vv][:,:,0] # F,R
    pp = 68
    med, *conf = np.percentile(yy, (50, 50-pp/2, 50+pp/2), axis=-1)
    hh, = ax.plot(xx, med, label=params_cl[vv][target], color=colors[vv])
    handles.append(hh)
    ax.fill_between(xx, *conf, alpha=0.25, color=colors[vv])

ax.legend(handles=handles, loc='lower right', title = plot.PARAM_KEYS[target], ncols=3)

In [ ]:


def plot_anis_vars(target, xx, yy_cl, params_cl, colors, parvars):
    xlabel = plot.LABEL_GW_FREQUENCY_YR
    ylabel = plot.LABEL_CLC0
    fig, ax = plot.figax_single(
    )
    ax.set_xlabel(xlabel)
    ax.set_ylabel(ylabel)
    
    handles = draw_anis_vars(ax, target, xx, yy_cl, params_cl, colors, parvars)
    
    ax.legend(handles=handles, loc='lower right', title = plot.PARAM_KEYS[target], ncols=3)

    return fig

def draw_anis_vars(ax, target, xx, yy_cl, params_cl, colors, parvars):
    ll=1
    handles = []
    for vv, var in enumerate(parvars):
        yy = yy_cl[vv][:,:,ll]/yy_cl[vv][:,:,0] # F,R
        pp = 68
        med, *conf = np.percentile(yy, (50, 50-pp/2, 50+pp/2), axis=-1)
        hh, = ax.plot(xx, med, label=params_cl[vv][target], color=colors[vv])
        handles.append(hh)
        ax.fill_between(xx, *conf, alpha=0.25, color=colors[vv])
    return handles
    

# Vary gsmf_phi0

In [ ]:
target = 'gsmf_phi0'
parvars = [0,10,20]
yy_cl2, xx_fobs, params_cl2 = build_anis_arrays(target, parvars=parvars)


In [ ]:
colors = cmap_Greens(np.linspace(0,1,len(parvars)))
fig = plot_anis_vars(target, xx_fobs*YR, yy_cl2, params_cl2,
                     colors=colors, parvars=parvars)

In [ ]:
xlabel = plot.LABEL_GW_FREQUENCY_YR
ylabel = r"$C_1/C_0$"
targets = [
    'gsmf_phi0', 'gsmf_mchar0_log10', 
      'mmb_mamp_log10',  'mmb_scatter_dex',
      'hard_time',  'hard_gamma_inner']
cmaps = [
    cmap_Greens, cmap_Greens,
    cmap_Oranges, cmap_Oranges,
    cmap_Blues, cmap_Blues
]
parvars = [0,10,20]

fig, axs = plot.figax_single(ncols=2, nrows=3)
fig.text(0.5, 0.08, xlabel, va='center', ha='center')
fig.text(0.08, 0.5, ylabel, va='center', ha='center', rotation='vertical')

for ii, ax in enumerate(axs.flatten()):
    filename = '/Users/emigardiner/GWs/holodeck/output/figures/bigplots/anis/anis'
    filename = detstats.append_filename(filename,
                                        bgl=BGL)
    filename += f"_pv3"
    filename += f".npz"
    if BUILD_ARRAYS:
        yy_cl, xx_fobs, params_cl = build_anis_arrays(targets[ii], parvars=parvars)
        np.savez(filename, yy_cl=yy_cl, xx_fobs=xx_fobs, params_cl=params_cl)
    else:
        file = np.load(filename)
        yy_cl = file['yy_cl']
        params_cl = file['params_cl']
        xx_fobs = file['xx_fobs']
        file.close()

    colors = cmaps[ii](np.linspace(0,1,len(parvars)))
    handles = draw_anis_vars(ax, targets[ii], xx_fobs*YR, yy_cl, params_cl, colors, parvars)


In [ ]:
BUILD_ARRAYS=True

In [ ]:
xlabel = plot.LABEL_GW_FREQUENCY_YR
ylabel = r"$C_1/C_0$"
targets = [
    'gsmf_phi0', 'gsmf_mchar0_log10', 
      'mmb_mamp_log10',  'mmb_scatter_dex',
      'hard_time',  'hard_gamma_inner']
cmaps = [
    cmap_Greens, cmap_Greens,
    cmap_Oranges, cmap_Oranges,
    cmap_Blues, cmap_Blues
]
parvars = [0,10,20]

fig, axs = plot.figax_single(ncols=2, nrows=3, height=7)
fig.text(0.5, 0.08, xlabel, va='center', ha='center')
fig.text(0.08, 0.5, ylabel, va='center', ha='center', rotation='vertical')

for ii, ax in enumerate(axs.flatten()):
    filename = '/Users/emigardiner/GWs/holodeck/output/anatomy_redz/figdata/anis/anis'
    filename = detstats.append_filename(filename,
                                        bgl=BGL)
    filename += f"_pv3"
    filename += f".npz"
    if BUILD_ARRAYS:
        yy_cl, xx_fobs, params_cl = build_anis_arrays(targets[ii], parvars=parvars)
        np.savez(filename, yy_cl=yy_cl, xx_fobs=xx_fobs, params_cl=params_cl)
    else:
        file = np.load(filename)
        yy_cl = file['yy_cl']
        params_cl = file['params_cl']
        xx_fobs = file['xx_fobs']
        file.close()

    colors = cmaps[ii](np.linspace(0,1,len(parvars)))
    handles = draw_anis_vars(ax, targets[ii], xx_fobs*YR, yy_cl, params_cl, colors, parvars)


In [ ]:
BUILD_ARRAYS=False



xlabel = plot.LABEL_GW_FREQUENCY_YR
ylabel = r"$C_1/C_0$"
targets = [
    'gsmf_phi0', 'gsmf_mchar0_log10', 
      'mmb_mamp_log10',  'mmb_scatter_dex',
      'hard_time',  'hard_gamma_inner']
cmaps = [
    cmap_Greens, cmap_Greens,
    cmap_Oranges, cmap_Oranges,
    cmap_Blues, cmap_Blues
]
parvars = [0,10,20]



fig, axs = plot.figax_single(ncols=2, nrows=3, height=7,
                             sharey=True, sharex=True)
fig.text(0.5, 0.08, xlabel, va='center', ha='center')
fig.text(0.05, 0.5, ylabel, va='center', ha='center', rotation='vertical')

for ii, ax in enumerate(axs.flatten()):
    filename = '/Users/emigardiner/GWs/holodeck/output/anatomy_redz/figdata/anis/anis'
    filename = detstats.append_filename(filename,
                                        bgl=BGL)
    filename += f"_pv3"
    filename += f".npz"
    if BUILD_ARRAYS:
        yy_cl, xx_fobs, params_cl = build_anis_arrays(targets[ii], parvars=parvars)
        np.savez(filename, yy_cl=yy_cl, xx_fobs=xx_fobs, params_cl=params_cl)
    else:
        file = np.load(filename, allow_pickle=True)
        yy_cl = file['yy_cl']
        params_cl = file['params_cl']
        xx_fobs = file['xx_fobs']
        file.close()
    colors = cmaps[ii](np.linspace(0,1,len(parvars)))
    handles = draw_anis_vars(ax, targets[ii], xx_fobs*YR, yy_cl, params_cl, colors, parvars)
fig.subplots_adjust(wspace=0, hspace=0)
